In [1]:
# Imports
import pandas as pd
import numpy as np

In [2]:
# Definitions
filename = r"../../data/int_agreement (cleaned).xlsx"
sheet_names = ["Arto", "Evanfiya", "James", "Juho", "Sami"]


def load_spreadsheet(filepath):
    sheets = pd.read_excel(filename, sheet_name=sheet_names)
    return sheets

def label_data(df, slicer, predicate, label="label", use_all=True):
    reducer = all if use_all else any
    labels = (df.iloc[slicer] # Pick relevant data
        .apply(predicate) # Apply predicate
        .apply(reducer, axis=1) # Reduce to single boolean column
        .apply(lambda label : "yes" if label else "no")) # Convert back to yes/no
    labels.name = label
    return labels

# Filters
is_null = lambda x: x.isnull()

In [3]:
sheets = load_spreadsheet(filename)
df = pd.concat(sheets.values(), axis=0).dropna(axis=0, subset="title")
#df.reset_index(inplace=True, drop=True)

In [4]:
slice_obj = np.s_[:, 10:14]
pred = lambda s: s.str.lower() == "yes"
label = "All metrics are correct (Yes/No)"

result1 = pd.concat([df, label_data(df, slice_obj, pred, label)], axis=1)
#result1.head()

In [5]:
#result1.to_csv("out.csv", sep=";", columns=["title", "problemDescription", "exampleSolution", label], index=False)

In [6]:
stats = [10, 11, 12, 13, 18]

count_yes = (result1.iloc[:, stats] == "yes").sum()
count_no = len(result1) - count_yes
percentages = count_yes / len(result1)

pd.DataFrame(data={"Correct (Yes)": count_yes, "Incorrect (Partially/No)": count_no, "Percentage of correct": percentages})

#df[label].where(lambda x : x == "yes").dropna().count() / len(df[label])

,Correct (Yes),Incorrect (Partially/No),Percentage of correct
The exercise description was clear (Yes/Partially/No),153,12,0.927273
The exercise description matched the selected theme (Yes/Partially/No),162,3,0.981818
The exercise description matched the selected topic (Yes/Partially/No),160,5,0.969697
The exercise description matched the selected concept (Yes/No),145,20,0.878788
All metrics are correct (Yes/No),128,37,0.775758
